In [1]:
import pickle
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")
from datetime import datetime


from queries import final_features
from score_funcs_copy import Score
sc = Score()

/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


In [2]:
data_params = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/data_params_v2.pkl')
model = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/redec_3.pkl')
transformer = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/scale_train_object.pkl')

In [3]:
pdf = pd.DataFrame(data = transformer.feature_names_in_, columns = ['feature'])
pdf['mean'] = transformer.mean_
pdf['std'] = np.sqrt(transformer.var_)

In [4]:
pdf = pdf.merge(data_params,left_on='feature',right_on = 'feature')
pdf

,feature,mean,std,median,lower_limit,upper_limit
0,distinct_debit_txns_1m,78.021832,128.478779,40.000000,1.0000,815.000000
1,median_amount_debited_2m,22.615685,40.766923,11.590000,0.0000,263.691700
2,ratio_debit_credit_2m,1.115814,0.413614,1.021660,0.4622,3.647596
3,ratio_ach_credit_freq_total_credit_1m,0.516212,0.273880,0.522774,0.0000,0.963411
4,ratio_ach_debit_credit_freq_1m,1.296399,1.769703,0.750000,0.0000,11.000000
5,median_running_balance_2nd_m,2695.882562,4683.993809,971.005000,-240.9432,28671.967000
6,ratio_revenue_1m_2m,13.258101,84.348046,0.998223,0.0000,772.812175
7,distinct_cashins_1m_prev,2.009052,1.905494,1.000000,0.0000,12.000000
8,ratio_amt_drawn_mom,330.739217,1200.548595,0.000000,0.0000,8115.000000
9,ratio_amt_cashin_mom,261.157620,745.312731,1.161163,0.0000,4497.300000


In [5]:
df = pd.read_pickle(f'/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/data/dataset_2.pkl')

In [6]:
df.shape

(4925, 125)

In [7]:
df.head()

,business_id,due_date,DPD_plus_15_next,DPD_plus_3_last_2,DPD_plus_15_prev,emi_no,DPD_plus_3,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,...,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom,pilot_flag,pilot_period_flag,fico_score,rs2_bin,rs2_score
0,c616b39b-aef7-4156-973a-08278aa74536,2023-04-03,0.0,0.0,0.0,1,0,0.0,1.0,0.0,...,0.0,0.357015,2884.000000,0.25,2.000000,0,0,749,5.0,971.0
1,cc861415-fd34-4654-8b97-dab0794938dc,2023-04-03,0.0,0.0,0.0,1,0,0.0,14.0,4.0,...,0.0,0.519896,2433.330000,0.50,2.000000,0,0,670,5.0,822.0
2,4d7e9b73-dd94-4fdc-9611-51ec641b8e19,2023-04-03,0.0,1.0,0.0,1,1,0.0,1.0,0.0,...,0.0,0.000000,2196.830000,0.00,2.000000,0,0,722,5.0,850.0
3,2e3c32e2-1f1b-4d0f-a7c2-8dcdb11c6317,2023-04-03,0.0,0.0,0.0,1,0,3.0,19.0,21.0,...,5.0,0.000000,0.949472,0.00,0.833333,0,0,721,5.0,795.0
4,57948378-e103-4f19-972e-a911082a16c2,2023-04-03,0.0,0.0,0.0,1,0,17.0,2.0,19.0,...,0.0,0.000000,363.000000,0.00,1.000000,0,0,652,3.0,551.0


In [8]:
df0 = sc.preprocess(df,sc.features,data_params,transformer)
df0 = sc.model_scoring_redec(df0,sc.features,model)

In [9]:
df0 = df0[~(df0['emi_no'].isin([1,6]))]

In [10]:
df0['DPD_plus_15_next'].value_counts()

0.0    2474
1.0     210
Name: DPD_plus_15_next, dtype: int64

In [11]:
X = df0.drop('DPD_plus_15_next', axis=1)
y = df0['DPD_plus_15_next']

In [12]:
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
X_train, X_test, y_train, y_test = train_test_split(df0.drop('DPD_plus_15_next', axis=1), df0['DPD_plus_15_next'], test_size=0.3, stratify=df0['DPD_plus_15_next'], random_state = 17)

In [13]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [14]:
X = X[sc.features]
X_train = X_train[sc.features]
X_train.head()

,distinct_debit_txns_1m,median_amount_debited_2m,ratio_debit_credit_2m,ratio_ach_credit_freq_total_credit_1m,ratio_ach_debit_credit_freq_1m,median_running_balance_2nd_m,ratio_revenue_1m_2m,distinct_cashins_1m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom
0,70.0,7.39,1.270083,0.608696,0.533333,2006.655,1.140402,1.0,0.566478,1.043956
1,125.0,0.00,0.935931,0.266667,0.000000,-197.390,0.062869,1.0,0.133289,634.990000
2,19.0,35.00,1.494456,0.800000,2.600000,10366.880,1.404023,1.0,0.000000,10.009003
3,134.0,0.00,1.018394,0.166667,1.166667,1097.065,0.697058,2.0,1.191849,0.865757
4,70.0,7.39,1.270083,0.608696,0.533333,0.590,0.017227,1.0,0.000000,0.520776


In [15]:
model

LogisticRegression(class_weight={0: 0.08, 1: 0.92}, random_state=17,
                   solver='liblinear')

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from skl2onnx.common import data_types as dt
from skl2onnx import to_onnx

In [17]:
scaler = StandardScaler()
lr = LogisticRegression(penalty= 'l2', class_weight={0: 0.08, 1: 0.92}, random_state=17,
                   solver='liblinear')

pipe = make_pipeline(scaler, lr)
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(class_weight={0: 0.08, 1: 0.92},
                                    random_state=17, solver='liblinear'))])

In [18]:
init_float = [(col, dt.FloatTensorType(shape=[None, 1])) for col in X_train.columns]
# initial_type = [('float_input', dt.FloatTensorType([None, 10]))]

onnx_model = to_onnx(
    pipe,
    initial_types=init_float,
    final_types=[
        ("label", dt.Int64TensorType([None])),
        ("probabilities", dt.FloatTensorType([None, 2])),
    ],
    options={LogisticRegression: {"zipmap": False}},
)

In [19]:
onnx_model

ir_version: 8
opset_import {
  domain: "ai.onnx.ml"
  version: 1
}
opset_import {
  domain: ""
  version: 9
}
producer_name: "skl2onnx"
producer_version: "1.14.1"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "distinct_debit_txns_1m"
    input: "median_amount_debited_2m"
    input: "ratio_debit_credit_2m"
    input: "ratio_ach_credit_freq_total_credit_1m"
    input: "ratio_ach_debit_credit_freq_1m"
    input: "median_running_balance_2nd_m"
    input: "ratio_revenue_1m_2m"
    input: "distinct_cashins_1m_prev"
    input: "ratio_amt_drawn_mom"
    input: "ratio_amt_cashin_mom"
    output: "concatenated"
    name: "FeatureVectorizer"
    op_type: "FeatureVectorizer"
    domain: "ai.onnx.ml"
    attribute {
      name: "inputdimensions"
      type: INTS
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
      ints: 1
    }
  }
  node {
    input: "concatenated"
    output: "variable"


In [20]:
import onnxruntime as rt
float_sess = rt.InferenceSession(onnx_model.SerializeToString())

print("The types of of the input are ...")
[(inp.name, inp.type, inp.shape) for inp in float_sess.get_inputs()]

The types of of the input are ...


[('distinct_debit_txns_1m', 'tensor(float)', [None, 1]),
 ('median_amount_debited_2m', 'tensor(float)', [None, 1]),
 ('ratio_debit_credit_2m', 'tensor(float)', [None, 1]),
 ('ratio_ach_credit_freq_total_credit_1m', 'tensor(float)', [None, 1]),
 ('ratio_ach_debit_credit_freq_1m', 'tensor(float)', [None, 1]),
 ('median_running_balance_2nd_m', 'tensor(float)', [None, 1]),
 ('ratio_revenue_1m_2m', 'tensor(float)', [None, 1]),
 ('distinct_cashins_1m_prev', 'tensor(float)', [None, 1]),
 ('ratio_amt_drawn_mom', 'tensor(float)', [None, 1]),
 ('ratio_amt_cashin_mom', 'tensor(float)', [None, 1])]

In [23]:
# with open("/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/models/rds1_model_xy.onnx", "wb") as f:
#     f.write(onnx_model.SerializeToString())